# AI Summit Workshop: Optimization in ML
## Session 1: Linear and Mixed Integer Programming
### Author: Prashanth Sriram

In this notebook, we will explore the locations of Sam's Club Stores and Distribution Centers.
We will then come up with an optimal alignment of Stores to Distribution Centers, so as to minimize the DC-Store transportation distance.

In [1]:
import pandas as pd
import numpy as np
import pulp as pulp
from ipywidgets import HTML
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Polyline

In [2]:
# Read DC 
sams_dc = pd.read_csv("/data/sams_dc.csv")
NUM_DC = sams_dc.shape[0]
print("Number of Sam's US DCs: " + str(NUM_DC))
sams_dc

Number of Sam's US DCs: 17


,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,DC_ZIP,DC_LONGITUDE,DC_LATITUDE
0,4792,Hattiesburg,MS,39401,-89.29034,31.32712
1,4889,New Braunfels,TX,78130,-98.12445,29.70300
2,6299,Buckeye,AZ,85326,-112.58378,33.37032
3,6492,North Canton,OH,44720,-81.40230,40.87590
4,6493,Ontario,CA,91761,-117.65090,34.06330
5,6494,Loveland,CO,80538,-105.07500,40.39780
6,6496,Kansas City,KS,66106,-94.62680,39.11550
7,6499,Villa Rica,GA,30180,-84.91910,33.73210
8,6596,Brownstown,MI,48193,-83.25720,42.12610
9,6698,Dayton,TX,77535,-94.88520,30.04660


In [3]:
# Read Club data
sams_club = pd.read_csv("/data/sams_club.csv")
NUM_CLUB = sams_club.shape[0]
print("Number of Sam's US Mainland Clubs: " + str(NUM_CLUB))
sams_club

Number of Sam's US Mainland Clubs: 600


,CLUB_LOC_ID,CLUB_LOC_CITY,CLUB_LOC_STATE,CLUB_ZIP,CLUB_LONGITUDE,CLUB_LATITUDE,NUM_LOADS
0,4041,Linden,NJ,7036,-74.231226,40.646898,791
1,4109,Bossier City,LA,71111,-93.706442,32.563612,568
2,4702,Friendswood,TX,77546,-95.156727,29.550006,666
3,4703,Albuquerque,NM,87114,-106.657313,35.210383,666
4,4704,Fresno,CA,93720,-119.794094,36.848661,884
...,...,...,...,...,...,...,...
595,8298,Joliet,IL,60436,-88.127917,41.516131,705
596,8299,Plano,TX,75075,-96.770484,33.001713,428
597,9475,Groveport,OH,43125,-82.887941,39.838052,1077
598,9639,Lumberton,NC,28360,-79.009825,34.669292,1249


In [5]:
600*17

10200

In [ ]:
# Visualize the DC and Club locations on the map
center = (39.240529, -94.464281)

m = Map(center=center, zoom=5)
m.layout.height = '600px'

for ind in sams_club.index: 
    club = CircleMarker(location=(sams_club['CLUB_LATITUDE'][ind], sams_club['CLUB_LONGITUDE'][ind]), weight = 2, radius = 4, color = 'red', opacity = 0.3)
    club.popup = HTML(value = sams_club['CLUB_LOC_CITY'][ind] + ', ' + sams_club['CLUB_LOC_STATE'][ind])
    m.add_layer(club)

for ind in sams_dc.index: 
    dc = CircleMarker(location=(sams_dc['DC_LATITUDE'][ind], sams_dc['DC_LONGITUDE'][ind]), weight = 2, radius = 8, color = 'blue')
    dc.popup = HTML(value = sams_dc['DC_LOC_CITY'][ind] + ', ' + sams_dc['DC_LOC_STATE'][ind])
    m.add_layer(dc)

    
#display(m)

In [4]:
# Read Distance Matrix
dist_matrix = pd.read_csv("/data/distance_matrix.csv")
dist_matrix['DISTANCE'] = dist_matrix['DISTANCE'].astype(np.int64)
print("Total possible combinations: " + str(dist_matrix.shape[0]))
dist_matrix

Total possible combinations: 10200


,CLUB_LOC_ID,CLUB_LOC_CITY,CLUB_LOC_STATE,CLUB_ZIP,CLUB_LONGITUDE,CLUB_LATITUDE,NUM_LOADS,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,DC_ZIP,DC_LONGITUDE,DC_LATITUDE,DISTANCE
0,4041,Linden,NJ,7036,-74.231226,40.646898,791,4792,Hattiesburg,MS,39401,-89.29034,31.32712,1057
1,4041,Linden,NJ,7036,-74.231226,40.646898,791,4889,New Braunfels,TX,78130,-98.12445,29.70300,1539
2,4041,Linden,NJ,7036,-74.231226,40.646898,791,6299,Buckeye,AZ,85326,-112.58378,33.37032,2156
3,4041,Linden,NJ,7036,-74.231226,40.646898,791,6492,North Canton,OH,44720,-81.40230,40.87590,375
4,4041,Linden,NJ,7036,-74.231226,40.646898,791,6493,Ontario,CA,91761,-117.65090,34.06330,2402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,8799,Perris,CA,92570,-117.228600,33.782500,984,8231,Taylor,PA,18517,-75.70660,41.39480,2310
10196,8799,Perris,CA,92570,-117.228600,33.782500,984,8232,Greenfield,IN,46140,-85.76940,39.78500,1779
10197,8799,Perris,CA,92570,-117.228600,33.782500,984,8234,Searcy,AR,72143,-91.73370,35.24680,1451
10198,8799,Perris,CA,92570,-117.228600,33.782500,984,8235,Sanger,TX,76266,-97.17390,33.36320,1153


## MIP Formulation

"Align each Sam's Club to one DC, such that the total truck load distance is minimized"

Now, structure this in terms of:
1. Sets
2. Constants
3. Decision Variables
4. Objective Function
5. Constraints

Our problem can be formulated thus:

1. Sets
 * Clubs  i: 1..C
 * DCs  j: 1..D
2. Constants
 * Distance Matrix $M_{ij}$
 * Club Loads $L_{i}$
3. Decision Variables
 * Club to DC Alignment Boolean Variables $X_{ij} \in \{0,1\}$
4. Objective Function (Minimize the total truck-load distance for the select Club-DC alignments) <br>
    $Min \sum \limits _{j=1} ^{D} \sum \limits _{i=1} ^{C} L_{i}*M_{ij}*X_{ij} $ <br>
5. Constraints
 * Each club can only be aligned to one DC <br>
    $\sum \limits _{j=1} ^{D} X_{ij} = 1 \; \forall \; i$

In [6]:
club_ids = sams_club.CLUB_LOC_ID.to_list()
dc_ids = sams_dc.DC_LOC_ID.to_list()

club_loads = dict(zip(club_ids, sams_club.NUM_LOADS.to_list()))

In [7]:
dist_matrix.sort_values(by = ['CLUB_LOC_ID', 'DC_LOC_ID'], inplace = True)
dist_matrix.head(20)

dist_matrix_list = [dist_matrix[(dist_matrix['CLUB_LOC_ID'] == club)].DISTANCE.to_list() for club in club_ids]
dist_matrix_dict = pulp.makeDict([club_ids, dc_ids], dist_matrix_list)


In [8]:
prob = pulp.LpProblem("ClubDC_Alignment", pulp.LpMinimize)
club_dc = pulp.LpVariable.dicts("ClubDC", (club_ids, dc_ids), cat='Binary')

club_dc_list = [(club, dc) for club in club_ids for dc in dc_ids]
prob += pulp.lpSum([dist_matrix_dict[club][dc]*club_loads[club]*club_dc[club][dc] for club, dc in club_dc_list]), "Objective_Function"


In [9]:
for club in club_ids:
    prob += pulp.lpSum(club_dc[club][dc] for dc in dc_ids) == 1, "Club_%s_To_One_DC"%club


In [10]:
# The problem is solved using PuLP's choice of Solver
p = prob.solve()

# The status of the solution is printed to the screen
print("Status: " + str(pulp.LpStatus[prob.status]))

# The minimum value of truck load miles achievable is
print("\nOptimized Objective Function Value: " + str(prob.objective.value()))

Status: Optimal

Optimized Objective Function Value: 49472780.0


In [11]:
alignments = [v.varValue for v in prob.variables()]
dist_matrix['ALIGNMENT'] = alignments
dist_matrix.head(20)

,CLUB_LOC_ID,CLUB_LOC_CITY,CLUB_LOC_STATE,CLUB_ZIP,CLUB_LONGITUDE,CLUB_LATITUDE,NUM_LOADS,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,DC_ZIP,DC_LONGITUDE,DC_LATITUDE,DISTANCE,ALIGNMENT
0,4041,Linden,NJ,7036,-74.231226,40.646898,791,4792,Hattiesburg,MS,39401,-89.29034,31.32712,1057,0.0
1,4041,Linden,NJ,7036,-74.231226,40.646898,791,4889,New Braunfels,TX,78130,-98.12445,29.70300,1539,0.0
2,4041,Linden,NJ,7036,-74.231226,40.646898,791,6299,Buckeye,AZ,85326,-112.58378,33.37032,2156,0.0
3,4041,Linden,NJ,7036,-74.231226,40.646898,791,6492,North Canton,OH,44720,-81.40230,40.87590,375,0.0
4,4041,Linden,NJ,7036,-74.231226,40.646898,791,6493,Ontario,CA,91761,-117.65090,34.06330,2402,0.0
5,4041,Linden,NJ,7036,-74.231226,40.646898,791,6494,Loveland,CO,80538,-105.07500,40.39780,1611,0.0
6,4041,Linden,NJ,7036,-74.231226,40.646898,791,6496,Kansas City,KS,66106,-94.62680,39.11550,1084,0.0
7,4041,Linden,NJ,7036,-74.231226,40.646898,791,6499,Villa Rica,GA,30180,-84.91910,33.73210,756,0.0
8,4041,Linden,NJ,7036,-74.231226,40.646898,791,6596,Brownstown,MI,48193,-83.25720,42.12610,478,0.0
9,4041,Linden,NJ,7036,-74.231226,40.646898,791,6698,Dayton,TX,77535,-94.88520,30.04660,1369,0.0


In [12]:
# Visualize the solution on the map
center = (39.240529, -94.464281)

m = Map(basemap=basemap_to_tiles(basemaps.CartoDB.Positron), center=center, zoom=5)
m.layout.height = '600px'

for ind in sams_club.index: 
    club = CircleMarker(location=(sams_club['CLUB_LATITUDE'][ind], sams_club['CLUB_LONGITUDE'][ind]), weight = 2, radius = 4, color = 'red', opacity = 0.3)
    club.popup = HTML(value = sams_club['CLUB_LOC_CITY'][ind] + ', ' + sams_club['CLUB_LOC_STATE'][ind])
    m.add_layer(club)

for ind in sams_dc.index: 
    dc = CircleMarker(location=(sams_dc['DC_LATITUDE'][ind], sams_dc['DC_LONGITUDE'][ind]), weight = 2, radius = 8, color = 'blue')
    dc.popup = HTML(value = sams_dc['DC_LOC_CITY'][ind] + ', ' + sams_dc['DC_LOC_STATE'][ind])
    m.add_layer(dc)

sel_alignments = dist_matrix[dist_matrix['ALIGNMENT'] == 1].reset_index()

for ind in sel_alignments.index: 
    line = Polyline(locations = [[sel_alignments['CLUB_LATITUDE'][ind], sel_alignments['CLUB_LONGITUDE'][ind]], [sel_alignments['DC_LATITUDE'][ind], sel_alignments['DC_LONGITUDE'][ind]]], weight = 1, color = 'green', opacity = 0.3)
    m.add_layer(line)
    
#display(m)

In [13]:
loads_per_dc = sel_alignments.groupby(['DC_LOC_ID', 'DC_LOC_CITY', 'DC_LOC_STATE']).agg(NUM_CLUBS = ('CLUB_LOC_ID', len), NUM_LOADS = ('NUM_LOADS', np.sum)).reset_index()
loads_per_dc

,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,NUM_CLUBS,NUM_LOADS
0,4792,Hattiesburg,MS,19,14589
1,4889,New Braunfels,TX,24,18378
2,6299,Buckeye,AZ,20,15885
3,6492,North Canton,OH,33,21217
4,6493,Ontario,CA,37,30311
5,6494,Loveland,CO,35,20019
6,6496,Kansas City,KS,39,22687
7,6499,Villa Rica,GA,38,26542
8,6596,Brownstown,MI,24,14519
9,6698,Dayton,TX,25,19239


### Let us add a wrinkle to this problem

Suppose that the maximum number fo annual loads that a DC can handle is only 30000.

Adding this additional constraint will cause some DCs to select fewer clubs, and the other clubs will be re-aligned to another DC farther away, but in such a way that the total truck load distance is optimal under these constraints.

Constraints
 * Each club can only be aligned to one DC <br>
    $\sum \limits _{j=1} ^{D} X_{ij} = 1 \; \forall \; i$
 * Each dc has a total limit of 30000 loads <br>
    $\sum \limits _{i=1} ^{C} L_{i}*X_{ij} \le 30000 \; \forall \; j$

In [14]:
for dc in dc_ids:
    prob += pulp.lpSum(club_dc[club][dc]*club_loads[club] for club in club_ids) <= 30000, "DC_%s_Capacity"%dc


In [15]:
# The problem is solved using PuLP's choice of Solver
p = prob.resolve()

# The status of the solution is printed to the screen
print("Status: " + str(pulp.LpStatus[prob.status]))

# The minimum value of truck load miles achievable is
print("\nOptimized Objective Function Value: " + str(prob.objective.value()))

Status: Optimal

Optimized Objective Function Value: 50153119.0


In [16]:
alignments = [v.varValue for v in prob.variables()]
dist_matrix['ALIGNMENT'] = alignments
dist_matrix.head(20)

,CLUB_LOC_ID,CLUB_LOC_CITY,CLUB_LOC_STATE,CLUB_ZIP,CLUB_LONGITUDE,CLUB_LATITUDE,NUM_LOADS,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,DC_ZIP,DC_LONGITUDE,DC_LATITUDE,DISTANCE,ALIGNMENT
0,4041,Linden,NJ,7036,-74.231226,40.646898,791,4792,Hattiesburg,MS,39401,-89.29034,31.32712,1057,0.0
1,4041,Linden,NJ,7036,-74.231226,40.646898,791,4889,New Braunfels,TX,78130,-98.12445,29.70300,1539,0.0
2,4041,Linden,NJ,7036,-74.231226,40.646898,791,6299,Buckeye,AZ,85326,-112.58378,33.37032,2156,0.0
3,4041,Linden,NJ,7036,-74.231226,40.646898,791,6492,North Canton,OH,44720,-81.40230,40.87590,375,0.0
4,4041,Linden,NJ,7036,-74.231226,40.646898,791,6493,Ontario,CA,91761,-117.65090,34.06330,2402,0.0
5,4041,Linden,NJ,7036,-74.231226,40.646898,791,6494,Loveland,CO,80538,-105.07500,40.39780,1611,0.0
6,4041,Linden,NJ,7036,-74.231226,40.646898,791,6496,Kansas City,KS,66106,-94.62680,39.11550,1084,0.0
7,4041,Linden,NJ,7036,-74.231226,40.646898,791,6499,Villa Rica,GA,30180,-84.91910,33.73210,756,0.0
8,4041,Linden,NJ,7036,-74.231226,40.646898,791,6596,Brownstown,MI,48193,-83.25720,42.12610,478,0.0
9,4041,Linden,NJ,7036,-74.231226,40.646898,791,6698,Dayton,TX,77535,-94.88520,30.04660,1369,0.0


In [17]:
# Visualize the solution on the map
center = (39.240529, -94.464281)

m = Map(basemap=basemap_to_tiles(basemaps.CartoDB.Positron), center=center, zoom=5)
m.layout.height = '600px'

for ind in sams_club.index: 
    club = CircleMarker(location=(sams_club['CLUB_LATITUDE'][ind], sams_club['CLUB_LONGITUDE'][ind]), weight = 2, radius = 4, color = 'red', opacity = 0.3)
    club.popup = HTML(value = sams_club['CLUB_LOC_CITY'][ind] + ', ' + sams_club['CLUB_LOC_STATE'][ind])
    m.add_layer(club)

for ind in sams_dc.index: 
    dc = CircleMarker(location=(sams_dc['DC_LATITUDE'][ind], sams_dc['DC_LONGITUDE'][ind]), weight = 2, radius = 8, color = 'blue')
    dc.popup = HTML(value = sams_dc['DC_LOC_CITY'][ind] + ', ' + sams_dc['DC_LOC_STATE'][ind])
    m.add_layer(dc)

sel_alignments = dist_matrix[dist_matrix['ALIGNMENT'] == 1].reset_index()

for ind in sel_alignments.index: 
    line = Polyline(locations = [[sel_alignments['CLUB_LATITUDE'][ind], sel_alignments['CLUB_LONGITUDE'][ind]], [sel_alignments['DC_LATITUDE'][ind], sel_alignments['DC_LONGITUDE'][ind]]], weight = 1, color = 'green', opacity = 0.3)
    m.add_layer(line)
    
#display(m)

In [18]:
loads_per_dc = sel_alignments.groupby(['DC_LOC_ID', 'DC_LOC_CITY', 'DC_LOC_STATE']).agg(NUM_CLUBS = ('CLUB_LOC_ID', len), NUM_LOADS = ('NUM_LOADS', np.sum)).reset_index()
loads_per_dc

,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,NUM_CLUBS,NUM_LOADS
0,4792,Hattiesburg,MS,20,15244
1,4889,New Braunfels,TX,25,19109
2,6299,Buckeye,AZ,21,16619
3,6492,North Canton,OH,47,29476
4,6493,Ontario,CA,36,29577
5,6494,Loveland,CO,35,20019
6,6496,Kansas City,KS,44,25763
7,6499,Villa Rica,GA,43,29967
8,6596,Brownstown,MI,26,15391
9,6698,Dayton,TX,26,20054


### More?

Suppose that we want to shutdown one of the DCs and re-align all the clubs accordingly. We want to see if the increase in transportation cost can be compensated by the decrease in facility cost.

This requires the addition of a binary variable representing whether a DC is in use or not; the constraint would be the DC selection binary has to be greater than or equal to **each** of the Club-DC alignment binary variables.

Then we can sum up the number of DCs selected and constrain it to an input number.

Constraints
 * Each club can only be aligned to one DC <br>
    $\sum \limits _{j=1} ^{D} X_{ij} = 1 \; \forall \; i$
 * Each dc has a total limit of 30000 loads <br>
    $\sum \limits _{i=1} ^{C} L_{i}*X_{ij} \le 30000 \; \forall \; j$
 * A dc has to be selected if any of the clubs is aligned to it <br>
    $S_{j} \ge X_{ij} \; \forall \; i,j $
 * The total number of selected DCs is equal to 16 <br>
    $\sum \limits _{j=1} ^D S_{j} = 16 $

In [19]:
dc_select = pulp.LpVariable.dicts("DCselect", (dc_ids), cat='Binary')

for dc in dc_ids:
    for club in club_ids:
        prob += pulp.lpSum(dc_select[dc] - club_dc[club][dc]) >= 0


In [20]:
prob += pulp.lpSum(dc_select[dc] for dc in dc_ids) == 16

In [21]:
# The problem is solved using PuLP's choice of Solver
p = prob.resolve()

# The status of the solution is printed to the screen
print("Status: " + str(pulp.LpStatus[prob.status]))

# The minimum value of truck load miles achievable is
print("\nOptimized Objective Function Value: " + str(prob.objective.value()))

Status: Optimal

Optimized Objective Function Value: 52016219.0


In [22]:
alignments = [v.varValue for v in prob.variables()]
alignments = alignments[0:(NUM_CLUB*NUM_DC)]
dist_matrix['ALIGNMENT'] = alignments
dist_matrix.head(20)

,CLUB_LOC_ID,CLUB_LOC_CITY,CLUB_LOC_STATE,CLUB_ZIP,CLUB_LONGITUDE,CLUB_LATITUDE,NUM_LOADS,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,DC_ZIP,DC_LONGITUDE,DC_LATITUDE,DISTANCE,ALIGNMENT
0,4041,Linden,NJ,7036,-74.231226,40.646898,791,4792,Hattiesburg,MS,39401,-89.29034,31.32712,1057,0.0
1,4041,Linden,NJ,7036,-74.231226,40.646898,791,4889,New Braunfels,TX,78130,-98.12445,29.70300,1539,0.0
2,4041,Linden,NJ,7036,-74.231226,40.646898,791,6299,Buckeye,AZ,85326,-112.58378,33.37032,2156,0.0
3,4041,Linden,NJ,7036,-74.231226,40.646898,791,6492,North Canton,OH,44720,-81.40230,40.87590,375,0.0
4,4041,Linden,NJ,7036,-74.231226,40.646898,791,6493,Ontario,CA,91761,-117.65090,34.06330,2402,0.0
5,4041,Linden,NJ,7036,-74.231226,40.646898,791,6494,Loveland,CO,80538,-105.07500,40.39780,1611,0.0
6,4041,Linden,NJ,7036,-74.231226,40.646898,791,6496,Kansas City,KS,66106,-94.62680,39.11550,1084,0.0
7,4041,Linden,NJ,7036,-74.231226,40.646898,791,6499,Villa Rica,GA,30180,-84.91910,33.73210,756,0.0
8,4041,Linden,NJ,7036,-74.231226,40.646898,791,6596,Brownstown,MI,48193,-83.25720,42.12610,478,0.0
9,4041,Linden,NJ,7036,-74.231226,40.646898,791,6698,Dayton,TX,77535,-94.88520,30.04660,1369,0.0


In [23]:
# Visualize the solution on the map
center = (39.240529, -94.464281)

m = Map(basemap=basemap_to_tiles(basemaps.CartoDB.Positron), center=center, zoom=5)
m.layout.height = '600px'

for ind in sams_club.index: 
    club = CircleMarker(location=(sams_club['CLUB_LATITUDE'][ind], sams_club['CLUB_LONGITUDE'][ind]), weight = 2, radius = 4, color = 'red', opacity = 0.3)
    club.popup = HTML(value = sams_club['CLUB_LOC_CITY'][ind] + ', ' + sams_club['CLUB_LOC_STATE'][ind])
    m.add_layer(club)

for ind in sams_dc.index: 
    dc = CircleMarker(location=(sams_dc['DC_LATITUDE'][ind], sams_dc['DC_LONGITUDE'][ind]), weight = 2, radius = 8, color = 'blue')
    dc.popup = HTML(value = sams_dc['DC_LOC_CITY'][ind] + ', ' + sams_dc['DC_LOC_STATE'][ind])
    m.add_layer(dc)

sel_alignments = dist_matrix[dist_matrix['ALIGNMENT'] == 1].reset_index()

for ind in sel_alignments.index: 
    line = Polyline(locations = [[sel_alignments['CLUB_LATITUDE'][ind], sel_alignments['CLUB_LONGITUDE'][ind]], [sel_alignments['DC_LATITUDE'][ind], sel_alignments['DC_LONGITUDE'][ind]]], weight = 1, color = 'green', opacity = 0.3)
    m.add_layer(line)
    
#display(m)

In [24]:
loads_per_dc = sel_alignments.groupby(['DC_LOC_ID', 'DC_LOC_CITY', 'DC_LOC_STATE']).agg(NUM_CLUBS = ('CLUB_LOC_ID', len), NUM_LOADS = ('NUM_LOADS', np.sum)).reset_index()
loads_per_dc

,DC_LOC_ID,DC_LOC_CITY,DC_LOC_STATE,NUM_CLUBS,NUM_LOADS
0,4792,Hattiesburg,MS,30,22884
1,4889,New Braunfels,TX,26,19714
2,6299,Buckeye,AZ,21,16619
3,6492,North Canton,OH,47,29476
4,6493,Ontario,CA,36,29577
5,6494,Loveland,CO,35,20019
6,6496,Kansas City,KS,49,29001
7,6499,Villa Rica,GA,43,29967
8,6596,Brownstown,MI,26,15391
9,6698,Dayton,TX,27,21029


The framework for formulating and solving Linear Programming and Mixed Integer Programming problems is very powerful, and a variety of problems can be solved in this manner.

Even non-linear objective functions that can have a good approximation as a piecewise linear function with a small number of lines, can be solved as an MIP.

However, truly complex non-linear problems with a high dimensionality of decision variables need a different approach.